In [4]:
# Pasul 1: Instalarea pachetelor necesare
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers

In [9]:
# Pasul 2: Importarea bibliotecilor necesare
from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline
import re

In [10]:
# Pasul 3: Pregătirea documentului și a embedding-urilor
# Documentul considerat "adevărul absolut"
document_text = """
Kaufland își face aprovizionarea duminica.
Toți angajații sunt instruiți să respecte regulile de securitate.
Magazinul se deschide la ora 8 dimineața în fiecare zi.
"""

# Spargerea documentului în părți mai mici (chunk-uri)
def split_text(text, chunk_size=100, chunk_overlap=20):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - chunk_overlap):
        chunks.append(" ".join(words[i:i + chunk_size]))
    return chunks

chunks = split_text(document_text)

# Crearea embedding-urilor folosind SentenceTransformers
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
document_embeddings = model.encode(chunks)

# Crearea unui index FAISS pentru căutare rapidă
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(document_embeddings)

# Funcție pentru extragerea propoziției relevante
def extract_relevant_sentence(chunk, prompt):
    # Spargem chunk-ul în propoziții
    sentences = re.split(r'(?<=[.!?])\s+', chunk)
    # Generăm embedding-uri pentru fiecare propoziție
    sentence_embeddings = model.encode(sentences)
    # Generăm embedding-ul pentru prompt
    prompt_embedding = model.encode([prompt])
    # Căutăm propoziția cea mai apropiată
    index_sentences = faiss.IndexFlatL2(sentence_embeddings.shape[1])
    index_sentences.add(sentence_embeddings)
    distances, indices = index_sentences.search(prompt_embedding, k=1)
    return sentences[indices[0][0]]

In [11]:
# Pasul 4: Funcție pentru verificarea adevărului
def check_truth(prompt):
    # Generarea embedding-ului pentru prompt
    query_embedding = model.encode([prompt])

    # Găsirea chunk-ului cel mai apropiat
    distances, indices = index.search(query_embedding, k=1)
    most_relevant_chunk = chunks[indices[0][0]]

    # Crearea unui pipeline pentru inferență logică
    nli_model = pipeline("text-classification", model="facebook/bart-large-mnli")

    # Verificarea relației între chunk-ul documentului și prompt
    premise = most_relevant_chunk
    hypothesis = prompt
    inference = nli_model(f"Premise: {premise} Hypothesis: {hypothesis}")
    label = inference[0]['label']

    # Interpretarea rezultatului
    if label == "contradiction":
        # Extragem propoziția relevantă
        relevant_sentence = extract_relevant_sentence(most_relevant_chunk, prompt)
        return f"Fals: Prompt-ul contrazice documentul. Informația corectă este: '{relevant_sentence}'"
    elif label == "entailment":
        return "Adevărat: Prompt-ul este în conformitate cu documentul."
    else:
        return "Neclar: Relația dintre prompt și document nu este sigură."

In [12]:
# Pasul 5: Testarea funcționalității
prompt = "Kaufland își reface stoc-ul lunea."
result = check_truth(prompt)
print(result)

Fals: Prompt-ul contrazice documentul. Informația corectă este: 'Kaufland își face aprovizionarea duminica.'
